<a href="https://colab.research.google.com/github/tarpandas/sentiment-analysis-on-tweet-data/blob/main/sentimentic_tarpan's_copy_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download kazanova/sentiment140

 73% 59.0M/80.9M [00:01<00:01, 21.3MB/s]
100% 80.9M/80.9M [00:01<00:00, 49.4MB/s]


In [ ]:
! unzip sentiment140.zip

Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [ ]:
import pandas as pd
import numpy as np
import re
import string

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
cols = ['target','id','date','flag','user','text']
df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1' ,names=cols)

In [ ]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df.shape

(1600000, 6)

In [ ]:
df.isnull().any()

target    False
id        False
date      False
flag      False
user      False
text      False
dtype: bool

In [ ]:
df = df[['target','text']]
df.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
nltk.download('wordnet')
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = re.sub(r'\@\w+|\@', "", text)

    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stopwords] #remove stopwords
    ps = PorterStemmer()
    stemmed_words = [ps.stem(w) for w in tokens]
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in stemmed_words]
        
    return " ".join(lemma_words)

df['text'] = [clean_text(text) for text in df['text']]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
df.head()

,target,text
0,0,switchfoot that bummer shoulda got david carr ...
1,0,upset cant updat facebook text might cri resul...
2,0,kenichan dive mani time ball manag save 50 res...
3,0,whole bodi feel itchi like fire
4,0,nationwideclass behav im mad cant see


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

X = df['text']
Y = df['target']
countvector=TfidfVectorizer(sublinear_tf=True)
X=countvector.fit_transform(X)

In [ ]:
print(X)

  (0, 157929)	0.15587753816604233
  (0, 631014)	0.3502191394773273
  (0, 110927)	0.44214413494984517
  (0, 156846)	0.30740732790365805
  (0, 244416)	0.17008084181466654
  (0, 569647)	0.38537583961160926
  (0, 101240)	0.3273150276521766
  (0, 622700)	0.20315974742019183
  (0, 610632)	0.4870390182645749
  (1, 83496)	0.36899063495151885
  (1, 35557)	0.27283227572878344
  (1, 639262)	0.1894144105454469
  (1, 554145)	0.24331399382757396
  (1, 529720)	0.35515499082341817
  (1, 144439)	0.2829721300303142
  (1, 408171)	0.27587977259325475
  (1, 621375)	0.3022711171195184
  (1, 206949)	0.31224444399738466
  (1, 663385)	0.2796871240874307
  (1, 108213)	0.19188918464916543
  (1, 664000)	0.3287859968122214
  (2, 91213)	0.3573665773781524
  (2, 240869)	0.13503295189439646
  (2, 529490)	0.25437320838777033
  (2, 11658)	0.31017864976729104
  :	:
  (1599996, 629814)	0.6494918637689555
  (1599996, 676864)	0.5109123739906005
  (1599996, 462282)	0.2609874346193936
  (1599996, 290397)	0.3371895002342383
 

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=19)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
mnb = MultinomialNB()
y_pred = mnb.fit(x_train, y_train).predict(x_test)
print('Naive-Bayes Accuracy :',accuracy_score(y_test,y_pred))

Naive-Bayes Accuracy : 0.75835


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[126976,  32809],
       [ 44519, 115696]])

In [ ]:
import pickle
file = open('model.pkl', 'wb')
pickle.dump(mnb,file)

## Accuracy test on 25% data

In [ ]:
df25 = df.loc[600000:1000000,:]
df25.shape

(400001, 2)

In [ ]:
X = df25['text']
Y = df25['target']
countvector=CountVectorizer(ngram_range=(1, 1))
X=countvector.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

In [ ]:
#Testing Naive-bayes on 25% data

mnb25 = MultinomialNB()
y_pred25 = mnb25.fit(x_train, y_train).predict(x_test)
print('Naive-Bayes Accuracy :',accuracy_score(y_test,y_pred25))
confusion_matrix(y_test, y_pred25)

Naive-Bayes Accuracy : 0.7854151823102211


array([[32760,  7291],
       [ 9876, 30074]])